In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('batch_1.csv')
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,266,TRANSFER,10000000.00,C1749184895,25494862.93,15494862.93,C1827220106,0.00,0.00,1
1,419,CASH_OUT,426323.38,C1078529507,426323.38,0.00,C1957980970,102216.61,528539.99,1
2,664,CASH_OUT,84771.55,C528998506,84771.55,0.00,C1971161098,1016318.82,1101090.37,1
3,506,CASH_OUT,10000000.00,C1180822515,10000000.00,0.00,C758711155,1920358.24,11920358.24,1
4,355,CASH_OUT,883366.69,C149173089,883366.69,0.00,C504582825,2029536.87,2912903.56,1


In [3]:
data['isFraud'].value_counts()

isFraud
1    8000
0    8000
Name: count, dtype: int64

In [4]:
data['type'].replace(['CASH_OUT', 'TRANSFER', 'PAYMENT', 'CASH_IN', 'DEBIT'],
                        list(range(5)), inplace=True)
# data['nameOrig'] = data['nameOrig'].apply(lambda x: int(x[1:]))
# data['nameDest'] = data['nameDest'].apply(lambda x: int(x[1:]))
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,266,1,10000000.00,C1749184895,25494862.93,15494862.93,C1827220106,0.00,0.00,1
1,419,0,426323.38,C1078529507,426323.38,0.00,C1957980970,102216.61,528539.99,1
2,664,0,84771.55,C528998506,84771.55,0.00,C1971161098,1016318.82,1101090.37,1
3,506,0,10000000.00,C1180822515,10000000.00,0.00,C758711155,1920358.24,11920358.24,1
4,355,0,883366.69,C149173089,883366.69,0.00,C504582825,2029536.87,2912903.56,1


In [5]:
# get the dummies and store it in a variable
dummies = pd.get_dummies(data[['nameOrig', 'nameDest']])
 
# Concatenate the dummies to original dataframe
data = pd.concat([data, dummies], axis='columns')
 
# drop the values
data = data.drop(['nameOrig', 'nameDest'], axis='columns')

In [6]:
data.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,nameOrig_C1000036340,nameOrig_C1000086512,...,nameDest_M992043620,nameDest_M992613667,nameDest_M9930262,nameDest_M994875026,nameDest_M994899742,nameDest_M995866768,nameDest_M99647526,nameDest_M996622886,nameDest_M99682381,nameDest_M999544978
0,266,1,10000000.00,25494862.93,15494862.93,0.00,0.00,1,False,False,...,False,False,False,False,False,False,False,False,False,False
1,419,0,426323.38,426323.38,0.00,102216.61,528539.99,1,False,False,...,False,False,False,False,False,False,False,False,False,False
2,664,0,84771.55,84771.55,0.00,1016318.82,1101090.37,1,False,False,...,False,False,False,False,False,False,False,False,False,False
3,506,0,10000000.00,10000000.00,0.00,1920358.24,11920358.24,1,False,False,...,False,False,False,False,False,False,False,False,False,False
4,355,0,883366.69,883366.69,0.00,2029536.87,2912903.56,1,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
X = data.drop(['isFraud'], axis=1)
y = data['isFraud']

#Standardization
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

2023-05-02 17:17:32.277639: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=31835))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [11]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

In [13]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

X_test = np.asarray(X_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

model.fit(X_train, y_train, epochs=20, batch_size=100)

Epoch 1/20
128/128 [==============================] - 1s 4ms/step - loss: 4.8543e-04 - accuracy: 0.9999
Epoch 2/20
128/128 [==============================] - 1s 4ms/step - loss: 4.8303e-04 - accuracy: 0.9999
Epoch 3/20
128/128 [==============================] - 1s 4ms/step - loss: 4.8065e-04 - accuracy: 0.9999
Epoch 4/20
128/128 [==============================] - 1s 4ms/step - loss: 4.7834e-04 - accuracy: 0.9999
Epoch 5/20
128/128 [==============================] - 1s 4ms/step - loss: 4.7605e-04 - accuracy: 0.9999
Epoch 6/20
128/128 [==============================] - 1s 4ms/step - loss: 4.7378e-04 - accuracy: 0.9999
Epoch 7/20
128/128 [==============================] - 1s 4ms/step - loss: 4.7159e-04 - accuracy: 0.9999
Epoch 8/20
128/128 [==============================] - 1s 4ms/step - loss: 4.6936e-04 - accuracy: 0.9999
Epoch 9/20
128/128 [==============================] - 1s 4ms/step - loss: 4.6718e-04 - accuracy: 0.9999
Epoch 10/20
128/128 [==============================] - 1s 4ms/st

In [14]:
y_hat = model.predict(X_test)
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

100/100 [==============================] - 0s 1ms/step


In [15]:
accuracy_score(y_test, y_hat)

0.56875

In [16]:
model.evaluate(X_test, y_test)

100/100 [==============================] - 0s 1ms/step - loss: 1.3716 - accuracy: 0.5688


[1.3715848922729492, 0.5687500238418579]